In [1]:
import pandas as pd
import numpy as np
import unicodedata
import datetime as dt

In [2]:
# Normalización de texto
def normalize_text(text):
    if pd.isna(text):
        return ""
    text = text.upper()
    text = ''.join(
        char for char in unicodedata.normalize('NFD', text) 
        if unicodedata.category(char) != 'Mn'
    )
    return text

# Limpieza de nombres específicos de municipios
def clean_municipio(text):
    replacements = {
        "BOGOTA, D.C": "BOGOTA",
        "BOGOTA D.C": "BOGOTA",
        "CARTAGENA DE INDIAS": "CARTAGENA"
    }
    return replacements.get(text, text)


In [3]:
#Población por ciudades
url = "Datasets/Poblacion_municipios.xlsx"
usecols = ['Municipio','Departamento', 'Poblacion Total']
dtype = {
    'Municipio': 'str',
    'Departamento': 'str',
    'Poblacion Total': 'float64'
}

# Leer el archivo CSV con las columnas necesarias y los tipos de datos especificados
pobla_municipio = pd.read_excel(url, usecols=usecols, dtype=dtype)


In [4]:
# Se aplica la normalización de datos de tipo texto en las columnas Municipio y departamento
pobla_municipio['Municipio'] = pobla_municipio['Municipio'].apply(normalize_text)
pobla_municipio['Departamento'] = pobla_municipio['Departamento'].apply(normalize_text)

In [5]:
# Filtrar las primeras 15 ciudades
departamentos = pobla_municipio['Departamento'].head(12)
departamentos = departamentos.unique()

Temperaturas

In [6]:
# Temperaturas por departamento
# Crear un rango de fechas desde el 2007-01-01 hasta el 2024-12-31
Temp = pd.date_range(start='2007-01-01', end='2024-12-31', freq='D')

# Crear un DataFrame con estas fechas
Temp = pd.DataFrame(Temp, columns=['FechaObservacion'])
Temp['FechaObservacion'] = pd.to_datetime(Temp['FechaObservacion']).dt.strftime("%Y-%m-%d")
Temp['FechaObservacion'] = Temp['FechaObservacion'].str.strip()

In [7]:
for row in departamentos:
    departamento = row

    # Nombre del archivo basado en el departamento
    archivo_csv = f"Datasets/Temperatura {departamento}.csv"

    usecols = ['FechaObservacion','Municipio','Departamento', 'ValorObservado']
    dtype = {
        'FechaObservacion' : 'str',
        'Municipio': 'str',
        'Departamento': 'str',
        'ValorObservado': 'float64'
    }

    # Leer el archivo CSV
    df_temperatura = pd.read_csv(archivo_csv, usecols=usecols, dtype=dtype)

    # Normalizar valores de ciudad para comparar
    df_temperatura['Municipio'] = df_temperatura['Municipio'].apply(normalize_text)

    # Cambiar valores de ciudades registrada de manera diferente
    df_temperatura['Municipio'] = df_temperatura['Municipio'].replace(
        {'BOGOTA, D.C': 'BOGOTA',
         'BOGOTA D.C': 'BOGOTA',
        'CARTAGENA DE INDIAS': 'CARTAGENA'}
    )

    # Convertir 'FechaObservacion' a datetime64[ns] con el formato correcto
    df_temperatura['FechaObservacion'] = df_temperatura['FechaObservacion'].str.split(" ").str[0]
    df_temperatura['FechaObservacion'] = pd.to_datetime(df_temperatura['FechaObservacion']).dt.strftime("%Y-%m-%d")

    # Primera agrupacion por fecha
    df_temperatura = df_temperatura.groupby(by=["FechaObservacion"])["ValorObservado"].mean().reset_index()
    df_temperatura["FechaObservacion"] = df_temperatura["FechaObservacion"].str.strip()

    # Cambio de nombre de la variable de temperatura
    df_temperatura = df_temperatura.rename(columns={'ValorObservado':f'Temp_{departamento}'})

    # Union de Dataframe
    Temp = Temp.merge(df_temperatura,on='FechaObservacion',how='left')



In [8]:
# Extraer el año y el mes
Temp['Año'] = Temp['FechaObservacion'].str.split("-").str[0]
Temp['Mes'] = Temp['FechaObservacion'].str.split("-").str[1]

# Seleccionar columnas de temperatura
columnas_temperatura = [col for col in Temp.columns if col.startswith('Temp_')]

# Imputar valores faltantes con el promedio mensual y, si persisten, usar el mes anterior
for col in columnas_temperatura:
    # Paso 1: Imputar con el promedio mensual (por Año y Mes)
    Temp[col] = Temp.groupby(['Año', 'Mes'])[col].transform(lambda x: x.fillna(x.mean()))
    
    # Paso 2: Identificar meses completos sin datos y rellenarlos con el promedio del mes anterior
    # Calcular el promedio del mes anterior si existen NaN persistentes
    for i in range(1, len(Temp)):
        if pd.isna(Temp.loc[i, col]) and i > 0:
            Temp.loc[i, col] = Temp.loc[i - 1, col]

In [9]:
Temp.to_csv("Temperaturas_Dpto.csv",index=False)